In [1]:
# implemention of recurrent_neural_network
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 超参数
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
# 理解LSTM的例子


# 输入的input为，序列长度seq_len=5, 每次取的minibatch大小，batch_size=3, 数据向量维数=10（仍然为x的维度）。每次运行时取3个含有5个字的句子（且句子中每个字的维度为10进行运行）
input1 = torch.randn(5, 3, 10)

# 输入数据x的向量维数10，设定lstm隐藏层的特征维度20，此model用2个lstm
rnn = nn.LSTM(10, 20, 2)

# 初始化隐藏元和记忆元，通常它们的维度相同
# 2个LSTM层，batch_size = 3,隐藏层维度20
h_0 = torch.randn(2, 3, 20)
c_0 = torch.randn(2, 3, 20)

# 这里有2层lstm，output是最后一层lstm的每个词向量对应隐藏层的输出，与其层数无关，只与序列长度有关
# hn，cn是所有层最后一个隐藏元和记忆元的输出
output, (h_n, c_n) = rnn(input1, (h_0, c_0))
print(output.shape, h_n.shape, c_n.shape)

torch.Size([5, 3, 20]) torch.Size([2, 3, 20]) torch.Size([2, 3, 20])


In [6]:
# RNN (many to one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0)) # out:tensor of shape(batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :]) # -1代表只取最后一个输出o[n-1],因为o[n-1]中体现了网络对一整个序列的认识（提取的特征），故取o[n-1]。
        # 取到o[n-1]后，我们再通过一个全连接层将o[n-1]映射到一个维度为output_dim的向量，向量的每一维就是网络对每一类别的预测置信度。
        return out


In [7]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/2], Step [100/600], Loss: 0.6161
Epoch [1/2], Step [200/600], Loss: 0.3192
Epoch [1/2], Step [300/600], Loss: 0.1444
Epoch [1/2], Step [400/600], Loss: 0.0966
Epoch [1/2], Step [500/600], Loss: 0.2986
Epoch [1/2], Step [600/600], Loss: 0.0287
Epoch [2/2], Step [100/600], Loss: 0.1227
Epoch [2/2], Step [200/600], Loss: 0.0857
Epoch [2/2], Step [300/600], Loss: 0.0668
Epoch [2/2], Step [400/600], Loss: 0.0184
Epoch [2/2], Step [500/600], Loss: 0.0349
Epoch [2/2], Step [600/600], Loss: 0.0505
Test Accuracy of the model on the 10000 test images: 96.89 %
